In [6]:
import pandas as pd

YOLO_CARS_CLASSES = ["car", "bus", "truck"]

# YOLO - only cars

for wildlife it did not know the names of animals

In [17]:
path_to_csv = 'datasets_generated_ready_models/stylegan2/detected_objects_cars.csv'
df_stylegan2_cars = pd.read_csv(path_to_csv)
df_stylegan2_cars.head(3)

,Unnamed: 0,image_path,predicted_class,confidence
0,0,datasets_generated_ready_models/stylegan2/gene...,car,0.95
1,1,datasets_generated_ready_models/stylegan2/gene...,truck,0.61
2,2,datasets_generated_ready_models/stylegan2/gene...,car,0.46


Filter-out non-car objects, count unique image_paths.

In [20]:
df_stylegan2_cars['class'] = df_stylegan2_cars['predicted_class'].apply(lambda x: 'car' if x in YOLO_CARS_CLASSES else 'other')

df_stylegan2_cars.head(3)

,Unnamed: 0,image_path,predicted_class,confidence,class
0,0,datasets_generated_ready_models/stylegan2/gene...,car,0.95,car
1,1,datasets_generated_ready_models/stylegan2/gene...,truck,0.61,car
2,2,datasets_generated_ready_models/stylegan2/gene...,car,0.46,car


In [21]:
# count the number of cars detected by each model
# 1. filter non cars
# 2. count the number of unique image_path
df_stylegan2_cars_filtered = df_stylegan2_cars[df_stylegan2_cars['class'] == 'car']

print(f"Number of cars detected by stylegan2: {df_stylegan2_cars_filtered['image_path'].nunique()}")


Number of cars detected by stylegan2: 194


# Hypothesis testing

$\phi(x) = 1$ if at least one car was detected.

In [22]:
Sn_stylegan2_cars = df_stylegan2_cars_filtered['image_path'].nunique() / len(df_stylegan2_cars['image_path'].unique())

Sn_stylegan2_cars

0.9748743718592965

In [26]:
import numpy as np

def add_results_row(results, experiment_name, N, Sn, p0_to_test=[0.5, 0.6, 0.7, 0.8, 0.9]):
    results["Method"].append("YOLO")
    results["Experiment"].append(experiment_name)
    results["N"].append(N)
    results["Sn"].append(Sn)
    for p0 in p0_to_test:
        if Sn <= p0:
            results[f"p0={p0}"].append(0)
            continue
        delta = np.exp(-2*N*(Sn-p0)**2)
        results[f"p0={p0}"].append(round(1-delta, 3))
    p0_for_CI_095 = Sn - np.sqrt(np.log(0.05)/(-2*N))
    results["p0 for confidence=0.95"].append(round(p0_for_CI_095, 3))

results = {
    "Method": [],
    "Experiment": [],
    "N": [],
    "Sn" : [],
    "p0=0.5": [],
    "p0=0.6": [],
    "p0=0.7": [],
    "p0=0.8": [],
    "p0=0.9": [],
    "p0 for confidence=0.95": [],
}

add_results_row(results, "StyleGAN2 cars", 200, Sn_stylegan2_cars)

df_results = pd.DataFrame(results)
df_results.to_csv("confidence_matrix_YOLO_stylegan2.csv", index=False)
df_results

,Method,Experiment,N,Sn,p0=0.5,p0=0.6,p0=0.7,p0=0.8,p0=0.9,p0 for confidence=0.95
0,YOLO,StyleGAN2 cars,200,0.974874,1.0,1.0,1.0,1.0,0.894,0.888
